In [1]:
pip install kafka-python

Note: you may need to restart the kernel to use updated packages.


In [1]:
from kafka import KafkaConsumer, KafkaProducer
import json
import pandas as pd
#producer = KafkaProducer(bootstrap_servers='192.168.99.100:9092')

In [2]:
#cannes_csv = pd.read_csv("Cannes2_plus.csv", delimiter=",", engine='python')
#movie_csv['timestamp'] = movie_csv['timestamp'].astype(str)
#cannes_csv = cannes_csv.drop(['timestamp'], axis=1)
#cannes_json_convert = cannes_csv.to_json("cannes.json")

In [4]:
# To illustrate streaming we read csv file and then stream using kafka
train_csv = pd.read_csv('train.txt', sep=" ", header=None)
train_csv = train_csv.loc[:, train_csv.isnull().sum() < 0.8*train_csv.shape[0]]
maximum = train_csv.groupby(0).agg({1: 'max'})
maximum
train_csv = pd.merge(train_csv, maximum, on=[0])
train_csv[27] = train_csv['1_y'] - train_csv['1_x']
train_csv = train_csv.drop(columns = ['1_y'])
train_csv.columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1','s2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21', 'ttf']
data_json_convert = train_csv.to_json("training.json")

In [5]:
train_csv.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20631 entries, 0 to 20630
Data columns (total 27 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        20631 non-null  int64  
 1   cycle     20631 non-null  int64  
 2   setting1  20631 non-null  float64
 3   setting2  20631 non-null  float64
 4   setting3  20631 non-null  float64
 5   s1        20631 non-null  float64
 6   s2        20631 non-null  float64
 7   s3        20631 non-null  float64
 8   s4        20631 non-null  float64
 9   s5        20631 non-null  float64
 10  s6        20631 non-null  float64
 11  s7        20631 non-null  float64
 12  s8        20631 non-null  float64
 13  s9        20631 non-null  float64
 14  s10       20631 non-null  float64
 15  s11       20631 non-null  float64
 16  s12       20631 non-null  float64
 17  s13       20631 non-null  float64
 18  s14       20631 non-null  float64
 19  s15       20631 non-null  float64
 20  s16       20631 non-null  fl

In [6]:
train_csv.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s13,s14,s15,s16,s17,s18,s19,s20,s21,ttf
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,191
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,190
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,189
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,188
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,187


In [ ]:
#import json
from kafka import KafkaConsumer, KafkaProducer
import time
import pandas as pd
from pyspark.sql import SparkSession

KAFKA_TOPIC_NAME = "pred_model"
KAFKA_BOOTSTRAP_SERVER_CONN = "192.168.99.100:9092"

kafka_producer_object = KafkaProducer(bootstrap_servers=KAFKA_BOOTSTRAP_SERVER_CONN,
                                      value_serializer=lambda x: json.dumps(x).encode('utf-8'))

training_json = pd.read_json("training.json")
print(training_json.head(1))

traininglist= training_json.to_dict(orient="records")
print(traininglist[0])

for record in traininglist:
    print("Message to be send : ", record)
    kafka_producer_object.send(KAFKA_TOPIC_NAME,record)
    time.sleep(1)		

kafka_producer_object.close()

   id  cycle  setting1  setting2  setting3      s1      s2      s3      s4  \
0   1      1   -0.0007   -0.0004       100  518.67  641.82  1589.7  1400.6   

      s5  ...      s13      s14     s15   s16  s17   s18  s19    s20     s21  \
0  14.62  ...  2388.02  8138.62  8.4195  0.03  392  2388  100  39.06  23.419   

   ttf  
0  191  

[1 rows x 27 columns]
{'id': 1, 'cycle': 1, 'setting1': -0.0007, 'setting2': -0.0004, 'setting3': 100, 's1': 518.67, 's2': 641.82, 's3': 1589.7, 's4': 1400.6, 's5': 14.62, 's6': 21.61, 's7': 554.36, 's8': 2388.06, 's9': 9046.19, 's10': 1.3, 's11': 47.47, 's12': 521.66, 's13': 2388.02, 's14': 8138.62, 's15': 8.4195, 's16': 0.03, 's17': 392, 's18': 2388, 's19': 100, 's20': 39.06, 's21': 23.419, 'ttf': 191}
Message to be send :  {'id': 1, 'cycle': 1, 'setting1': -0.0007, 'setting2': -0.0004, 'setting3': 100, 's1': 518.67, 's2': 641.82, 's3': 1589.7, 's4': 1400.6, 's5': 14.62, 's6': 21.61, 's7': 554.36, 's8': 2388.06, 's9': 9046.19, 's10': 1.3, 's11': 47.47, 